In [ ]:
# importing qiskit modules

from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.circuit import QuantumCircuit,ParameterVector
from qiskit.transpiler import PassManager
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimator
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.providers.options import Options
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [ ]:

# importing necessary netket libraries
import flax
import flax.linen as nn
import netket as nk
from netket.operator.spin import sigmax,sigmaz
import jax.numpy as jnp

In [ ]:
# other useful libraries
import pickle
import sys
import time

from tqdm import tqdm
from scipy.sparse.linalg import eigsh
from scipy.optimize import minimize

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# function to generate Hamiltonian and caclculate energy

def generate_Hamiltonian(N,J,h,hi):

  # single body terms
  H = sum([-J*sigmax(hi,i) for i in range(N)])
  # two body terms of the Hamiltonian
  H += sum([-h*sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])

  hstrings = H.to_pauli_strings()
  pauliop = list(hstrings.operators)
  coeff =  list(hstrings.weights)
  hpauli = SparsePauliOp(pauliop,coeffs=coeff)

  return H, hpauli

def calc_Energy(H):
  sp_h = H.to_sparse()
  eig_vals,eig_vecs = eigsh(sp_h,k=2,which="SA") # selecting only 2 eigvals and eigvecs that having smallest algebraic eigenvalues
  Egs = eig_vals[0]
  gs_state = eig_vecs[0]

  return Egs

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    energy = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return energy

def build_callback(ansatz, hamiltonian, estimator, callback_dict):
    def callback(current_vector):
        # Keep track of the number of iterations
        callback_dict["iters"] += 1
        # Set the prev_vector to the latest one
        callback_dict["prev_vector"] = list(current_vector)
        # Compute the value of the cost function at the current vector
        # This adds an additional function evaluation
        current_cost = (
            estimator.run(ansatz, hamiltonian, parameter_values=current_vector).result().values[0]
        )
        callback_dict["cost_history"].append(current_cost)
        # Print to screen on single line
        sys.stdout.write("\rIters. done: {} [Current cost: {}]".format(callback_dict["iters"], current_cost))
        sys.stdout.flush()  # Flush the output buffer

    return callback

In [ ]:
# Construct HVA for TFIM 

def TFIM_HVA(hpauli:SparsePauliOp,reps:int=3,initial_state:QuantumCircuit=None,entanglement:str='circular'):
  
  num_qubits = hpauli.num_qubits # number of qubits to create ansatz
  qubits = list(range(num_qubits))

  betas = ParameterVector(name='beta',length=reps) # parameters of zz layer
  gammas = ParameterVector(name='gamma',length=reps) # parameters of rx layer

  ansatz = QuantumCircuit(num_qubits)
  if initial_state:
    ansatz.append(initial_state,range(num_qubits))
  else:
    ansatz.h(qubits) # initial state

  for rep in range(reps):
    ansatz.barrier()
    if entanglement == 'linear': # for open boundary condition
      ansatz.rzz(betas[rep],qubits[:-1],qubits[1:])
    elif entanglement == 'reverse_linear': # for open boundary condition
      ansatz.rzz(betas[rep],qubits[1:],qubits[:-1])
    else: # For Periodic Boundary condition
      ansatz.rzz(betas[rep],qubits,qubits[1:]+qubits[0:1])
    ansatz.barrier()
    ansatz.rx(gammas[rep],range(num_qubits)) # one qubit rotations

  return(ansatz)

In [ ]:
# TFI model parameters
N = 15
J = 1
h = 1

# generating hamiltonian and theoretical energy
chain = nk.graph.Chain(N) # Periodic Boundary Conditions
hi = nk.hilbert.Spin(s=1/2) ** N
H,hpauli = generate_Hamiltonian(N,J,h,hi)
Egs = calc_Energy(H)

In [ ]:
reps = [1,2,3,4,5,6,8,10] # depth of the quantum circuit
niter = 300 # maxiter value used

backend = BasicSimulator() # local simulator used as backend
options = Options(shots=10000) # setting number of measurements for estimation of observable
estimator = BackendEstimator(backend=backend,options=options) # initialisng estimator for estimating expectation value of energy

In [ ]:
res  = []
np.random.seed(42)
for rep in reps:
  ansatz = TFIM_HVA(hpauli,reps=rep)
  pm = generate_preset_pass_manager(target=backend.target, optimization_level=3)
  ansatz_ibm = pm.run(ansatz)
  ham_ibm = hpauli.apply_layout(ansatz_ibm.layout)

  # dictionary to store intermediate values
  callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],}

  # finding optimal variance
  num_params = ansatz.num_parameters
  x0 = 2 * np.pi *np.random.random(num_params)

  # optimization of parameters
  callback = build_callback(ansatz,hpauli, estimator, callback_dict)
  st = time.time()
  r = minimize(cost_func,x0,
    args=(ansatz, hpauli, estimator),
    method="cobyla",
    callback=callback,
    options={'maxiter':niter},)
  et = time.time()
  min,sec = divmod(et-st,60)

  # optimal energy calculation
  opt_ansatz = ansatz.assign_parameters(parameters = callback_dict['prev_vector'])
  opt_res = estimator.run(circuits=[opt_ansatz]*100, observables=[hpauli]*100)
  opt_energy = opt_res.result().values.mean()
  error = abs((opt_energy - Egs)/Egs) # relative error

  res_dict = {}

  res_dict['N'] = N
  res_dict['J'] = J
  res_dict['h'] = h
  res_dict['reps'] = reps
  res_dict['niter'] = niter
  res_dict['num_parameters'] = num_params
  res_dict['optmization_data'] = callback_dict
  res_dict['optimised_energy'] = opt_energy
  res_dict['relative_error'] = error
  res_dict['exec_time'] = f'{int(min)}:{int(sec)}'
  res.append(res_dict)
  print("optimized energy and relative error",opt_energy,error)


In [2]:
import qiskit,netket
print('qiskit:', qiskit.__version__)
print('netket:',netket.__version__)


qiskit: 1.0.2
netket: 3.10.2
